In [32]:
import csv
import pandas as pd

In [34]:
# Navigate to folder
%cd /Users/Ayaire/Documents/Python/Katie_Hobbs

/Users/Ayaire/Documents/Python/Katie_Hobbs


## What I'm trying to do...

Create an empty dictionary where I'll put records that match on first name and last name across the two data sources and meet match thresholds based on county and address info from the two files.

Steps
* Normalize capitalization of the columns that are strings.
* Create a new dictionary to put the records you'll want to keep and review
* Compare, row-by-row. If a row matxhes on first name AND last name then append it to the dictionary you've made
* Create a match score based on county and address fields




## First Tabe: Contacts from Voter File

You will need to export contacts from the Voter File and include the fields that you 
want to match onto your petion signer info.
In this example we use FirstName, LastName, Address Number, City, County, Zipcode
We had address entries with no freakin' commas, so we needed to use the following 
in the spreadsheet to extract the City

For the City
=TRIM(REGEXEXTRACT(C8,"\b(?:AVE |AVENUE | BLVD | CIR | CIRCLE | CT | COURT | DR |DRIVE |LN |PL |PLACE | PKWY | PARKWAY |RD |ROAD | SQ | ST |STREET | TRL | WAY )(.*)AZ"))

For the Street Address
=REGEXEXTRACT(C2,"\d{1,5}\s")

In [ ]:
# There is a strong chance you will need to make several exports from MyVoters
#     due to being limited on the number of records you can export from van per Export Request (100-200K)
# Use the following if you have a bunch of CSVs that have the same fields that you need to combine into one CSV
import os
import glob
import pandas as pd
#set working directory
#file path to the folder where your csv's are saved
os.chdir("/Users/Ayaire/Documents/Python/Katie_Hobbs")
#find all csv files in the folder
#use glob pattern matching -> extension = 'csv'
#save result in list -> all_filenames
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#if you want to check that the files are the csv's you want
# print(all_filenames)
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
#this will save them in the same folder you indicated above
combined_csv.to_csv( "Democrats in Phoenix Yavapai and Tucson.csv", index=False, encoding='utf-8-sig')

In [35]:
# Read the table into a dataframe
voter_file_contacts = pd.read_csv("Democrats in Phoenix Yavapai and Tucson.csv", low_memory=False)

In [21]:
# I guess it would be nice to know what the columns' data types are
my_columns = voter_file_contacts.columns.values.tolist()
my_columns_types = voter_file_contacts.dtypes.tolist()

# Convert lists to dictionary
column_types = {my_columns[i]: my_columns_types[i] for i in range(len(my_columns))} 
for key, value in column_types.items():
    print(key, ": OBJECT TYPE: ", value)

Voter File VANID : OBJECT TYPE:  int64
LastName : OBJECT TYPE:  object
FirstName : OBJECT TYPE:  object
MiddleName : OBJECT TYPE:  object
Suffix : OBJECT TYPE:  object
mAddress : OBJECT TYPE:  object
mCity : OBJECT TYPE:  object
mState : OBJECT TYPE:  object
mZip5 : OBJECT TYPE:  float64
mZip4 : OBJECT TYPE:  float64
StreetNo : OBJECT TYPE:  float64
StreetNoHalf : OBJECT TYPE:  object
StreetPrefix : OBJECT TYPE:  object
StreetName : OBJECT TYPE:  object
StreetType : OBJECT TYPE:  object
StreetSuffix : OBJECT TYPE:  object
AptType : OBJECT TYPE:  object
AptNo : OBJECT TYPE:  object
City : OBJECT TYPE:  object
State : OBJECT TYPE:  object
Zip5 : OBJECT TYPE:  int64
Zip4 : OBJECT TYPE:  float64
CountyName : OBJECT TYPE:  object
PreferredEmail : OBJECT TYPE:  object
PersonalEmail : OBJECT TYPE:  object
PartyName : OBJECT TYPE:  object


In [22]:
# Check out the first 5 rows of the dataframe
voter_file_contacts.head()

,Voter File VANID,LastName,FirstName,MiddleName,Suffix,mAddress,mCity,mState,mZip5,mZip4,...,AptType,AptNo,City,State,Zip5,Zip4,CountyName,PreferredEmail,PersonalEmail,PartyName
0,99820,French,David,Kent,NaN,4205 E Altadena Ave,Phoenix,AZ,85028.0,2232.0,...,NaN,NaN,Phoenix,AZ,85028,2232.0,Maricopa,davidfrench943@gmail.com,davidfrench943@gmail.com,Democrat
1,245159,Sugrue Ashley,Mary,Kathleen,NaN,3402 W Hearn Rd,Phoenix,AZ,85053.0,5689.0,...,NaN,NaN,Phoenix,AZ,85053,5689.0,Maricopa,mkmashfam3@msn.com,mkmashfam3@msn.com,Democrat
2,59735,Aaron,Carrie,Lynn,NaN,4832 E Exeter Blvd,Phoenix,AZ,85018.0,2950.0,...,NaN,NaN,Phoenix,AZ,85018,2950.0,Maricopa,carrieaaron@cox.net,carrieaaron@cox.net,Democrat
3,11936734,Cook,Richard,Lawrence,NaN,753 E Moreland St Apt 3,Phoenix,AZ,85006.0,2713.0,...,Apt,3,Phoenix,AZ,85006,2713.0,Maricopa,NaN,NaN,Democrat
4,11937823,Driver,Lashonda,Ann,NaN,14430 N 19th Ave Apt 252,Phoenix,AZ,85023.0,6739.0,...,Apt,252,Phoenix,AZ,85023,6739.0,Maricopa,NaN,NaN,Democrat


## Second Tabe: Petition Signers

You're going to want to make sure that you have a unique ID for each row from your table of petition signers
because you will want to be able to join back to it at the end. 

In [37]:
petition_signers = pd.read_csv("E-Qual As of 11_18_21.csv", low_memory=False)

In [38]:
# Check Column Names and Data Types
my_columns = petition_signers.columns.values.tolist()
my_columns_types = petition_signers.dtypes.tolist()
# Convert lists to dictionary
column_types = {my_columns[i]: my_columns_types[i] for i in range(len(my_columns))} 
for key, value in column_types.items():
    print(key, ": OBJECT TYPE: ", value)

Voter Last Name : OBJECT TYPE:  object
Voter First Name : OBJECT TYPE:  object
Voter Address : OBJECT TYPE:  object
City : OBJECT TYPE:  object
State : OBJECT TYPE:  object
Zip : OBJECT TYPE:  float64
Street Address : OBJECT TYPE:  float64
Voter Jurisdiction Name : OBJECT TYPE:  object
Voter Email Address : OBJECT TYPE:  object
Date Signed : OBJECT TYPE:  object
ID for Mapping : OBJECT TYPE:  float64
In VAN  : OBJECT TYPE:  bool
MVID/Notes : OBJECT TYPE:  object


In [40]:
# Take a quick look at how the columns look
petition_signers.head()

,Voter Last Name,Voter First Name,Voter Address,City,State,Zip,Street Address,Voter Jurisdiction Name,Voter Email Address,Date Signed,ID for Mapping,In VAN,MVID/Notes
0,UANIS,JAMES,312 W EDGEMONT AVE PHOENIX AZ 85003,PHOENIX,AZ,85003.0,312.0,Maricopa,jcuanis@mac.com,11/4/2021,1.0,False,NaN
1,KELCHNER,JOAN,318 W LYNWOOD ST PHOENIX AZ 85003,PHOENIX,AZ,85003.0,318.0,Maricopa,DrJ4Roosevelt@cox.net,10/14/2021,2.0,False,NaN
2,ROSE,ROBERT,27961 N VARNUM RD SAN TAN VALLEY AZ 85143,SAN TAN VALLEY,AZ,85143.0,27961.0,Pinal,NaN,11/18/2021,3.0,True,NaN
3,LONG,SANDRA,505 W LEWIS AVE PHOENIX AZ 85003,PHOENIX,AZ,85003.0,505.0,Maricopa,NaN,10/4/2021,4.0,False,NaN
4,MC GRATH,MARILYN,319 W VIRGINIA AVE PHOENIX AZ 85003,PHOENIX,AZ,85003.0,319.0,Maricopa,marilynaz821@gmail.com,10/4/2021,5.0,False,NaN


## Normalize, Narrow and Create Empty Comparison Fields

* Make capitalization and format standardized between the fields you'll compare
* Narrow DataFrames to only relevant columns
* Creare empty fields for filling later with comparison indicators

In [68]:
# The fields you will be using
column_names = ["FirstName", "LastName", "StreetAddress", "City", "Zip", "County", "State",
                 "Match_StAddy", "Match_City","Match_Zip",  "Match_County"]
# Create empty dataframes with only the columns you want
p_signers = pd.DataFrame(columns = column_names)
vf_contacts = pd.DataFrame(columns = column_names)

In [69]:
#petition_signers
# looks like you'll want to UPPER capitaliztion CountyName. 
# LastName, FirstName seem fine but might as well UPPEr them in case you didn't see some lower case instances.
# This creates new columns that match name and format of the Voter File records
p_signers['Signer_Identifier'] = petition_signers['ID for Mapping']
p_signers['FirstName'] = petition_signers['Voter First Name'].str.upper()
p_signers['LastName'] = petition_signers['Voter Last Name'].str.upper()
p_signers['StreetAddress'] = petition_signers['Street Address']
p_signers['City'] = petition_signers['City'].str.upper()
p_signers['Zip'] = petition_signers['Zip']
p_signers['County'] = petition_signers['Voter Jurisdiction Name'].str.upper()
p_signers['State'] = petition_signers['State']
# Check out the first 5 rows of the dataframe
p_signers.head()

,FirstName,LastName,StreetAddress,City,Zip,County,State,Match_Name,Match_StAddy,Match_City,Match_Zip,Match_County,Signer_Identifier
0,JAMES,UANIS,312.0,PHOENIX,85003.0,MARICOPA,AZ,NaN,NaN,NaN,NaN,NaN,1.0
1,JOAN,KELCHNER,318.0,PHOENIX,85003.0,MARICOPA,AZ,NaN,NaN,NaN,NaN,NaN,2.0
2,ROBERT,ROSE,27961.0,SAN TAN VALLEY,85143.0,PINAL,AZ,NaN,NaN,NaN,NaN,NaN,3.0
3,SANDRA,LONG,505.0,PHOENIX,85003.0,MARICOPA,AZ,NaN,NaN,NaN,NaN,NaN,4.0
4,MARILYN,MC GRATH,319.0,PHOENIX,85003.0,MARICOPA,AZ,NaN,NaN,NaN,NaN,NaN,5.0


In [70]:
#voter_file_contacts
# looks like you'll want to UPPER capitaliztion on LastName, FirstName, CountyName, 
vf_contacts['VF_ID'] = voter_file_contacts['Voter File VANID']
vf_contacts['FirstName'] = voter_file_contacts['FirstName'].str.upper()
vf_contacts['LastName'] = voter_file_contacts['LastName'].str.upper()
vf_contacts['StreetAddress'] = voter_file_contacts['StreetNo']
vf_contacts['City'] = voter_file_contacts['City'].str.upper()
vf_contacts['Zip'] = voter_file_contacts['Zip5']
vf_contacts['County'] = voter_file_contacts['CountyName'].str.upper()
vf_contacts['State'] = voter_file_contacts['State'].str.upper()
# Check out the first 5 rows of the dataframe
vf_contacts.head()

,FirstName,LastName,StreetAddress,City,Zip,County,State,Match_Name,Match_StAddy,Match_City,Match_Zip,Match_County,VF_ID
0,DAVID,FRENCH,4205.0,PHOENIX,85028,MARICOPA,AZ,NaN,NaN,NaN,NaN,NaN,99820
1,MARY,SUGRUE ASHLEY,3402.0,PHOENIX,85053,MARICOPA,AZ,NaN,NaN,NaN,NaN,NaN,245159
2,CARRIE,AARON,4832.0,PHOENIX,85018,MARICOPA,AZ,NaN,NaN,NaN,NaN,NaN,59735
3,RICHARD,COOK,753.0,PHOENIX,85006,MARICOPA,AZ,NaN,NaN,NaN,NaN,NaN,11936734
4,LASHONDA,DRIVER,14430.0,PHOENIX,85023,MARICOPA,AZ,NaN,NaN,NaN,NaN,NaN,11937823


## Now we try to compare the dataframes, line-by-line

We want to look at every row of vf_contacts and for each row compare to each row in p_signers.
If vf_contacts's FirstName, LastName, State match then append the following fields
to the new list/dictionary/df (whatever it ends up being)
        p_signers['Signer_Identifier']
        vf_contacts['VF_ID']
        vf_contacts['FirstName']
        vf_contacts['LastName']
        vf_contacts[State]
Then take the following conditional steps:
if vf_contacts['StreetAddress'] = p_signers['StreetAddress']
    then df["Match_StAddy"] == TRUE
if vf_contacts['City'] = p_signers['City']
    then df["Match_City"] == TRUE
vf_contacts['Zip'] = p_signers['Zip']
    then df["Match_Zip"] == TRUE
vf_contacts['County'] = p_signers['County']
    then df["Match_County"] == TRUE


In [ ]:
vf_contacts
p_signers

In [72]:
VFL = []
PSL = []

In [80]:
match = []
for row in vf_contacts:
    while vf_contacts['FirstName'] == p_signers['FirstName']:
        match.append(row)
        
        
  

ValueError: Can only compare identically-labeled Series objects

In [76]:
match = []
for row in vf_contacts:
    for r in p_signers:
        while vf_contacts['FirstName'] == p_signers['FirstName'] \
        and vf_contacts['LastName'] in p_signers['LastName'] \
        and vf_contacts['State'] in p_signers['State']:
            match.append(row)
            
    and vf_contacts['City'] in p_signers['City'] \
    and vf_contacts['Zip'] in p_signers['Zip'] \
    and vf_contacts['StreetAddress'] in p_signers['StreetAddress']:
        match.append(row)
        
        
        
    

TypeError: unhashable type: 'Series'

In [ ]:
VFL = []
PSL = []

# For row in VF
#    if FirstName = PS.FirstName and LastName = PS.LastName
# 

# Petition Signers
with open("E-Qual As of 11_18_21.csv") as PSL:
    csv_PSL = csv.reader(PSL)
    for row in csv_PSL:
        PSL.append(tuple(row[0:3]))

# Your voter file CSV will have the most records so it should be faster if you 
#    compare the smaller CSV's records to it
with open("Democrats in Phoenix Yavapai and Tucson.csv") as CSV1:
    csv_CSV1 = csv.reader(CSV1)
    for row in csv_CSV1:
        CSV1_tuples.append(tuple(row[0:3]))
        if tuple(row[0:3]) in CSV2_tuples:
            print(( row[0:3] ), "In both DB1 & DB2")
        if tuple(row[1:3]) in CSV2_tuples:
            print(( row[0:3] ), "Wrong ID")


In [ ]:
"Democrats in Phoenix Yavapai and Tucson"
"E-Qual As of 11_18_21.csv"

In [ ]:
CSV1_tuples = []
CSV2_tuples = []

with open("DB1.csv") as CSV2:
    csv_CSV2 = csv.reader(CSV2)
    for row in csv_CSV2:
        CSV2_tuples.append(tuple(row[0:3]))

with open("DB2.csv") as CSV1:
    csv_CSV1 = csv.reader(CSV1)
    for row in csv_CSV1:
        CSV1_tuples.append(tuple(row[0:3]))
        if tuple(row[0:3]) in CSV2_tuples:
            print(( row[0:3] ), "In both DB1 & DB2")
        if tuple(row[1:3]) in CSV2_tuples:
            print(( row[0:3] ), "Wrong ID")


In [ ]:
def get_csv_data(csv_file, row, cell=None):
    """
    :param csv_file: Name of csv file
    :param row: Row number that you want( counting starts from top to bottom)
    :param cell: cell number that you want(counting starts from left to right)
    If you give a cell number, the content of that cell will be returned.
    If cell =
    :return: cell content
    """
    ls = []
    with open(csv_file, newline='') as csvfile:
        csv_file = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for rows in csv_file:
            ls.append(str(rows[0]).split(","))
        if cell is not None:
            return re.sub(r'\W+', '', str(ls[row-1]).split(",")[cell-1])
        else:
            return ls[row-1]

print(get_csv_data('csv1.csv', 2, 2)) #get row 2, cell 2 from csv1 -> returns John
print(get_csv_data('csv1.csv', 2)) #get row 2 from csv1 -> returns a list with all values from the row: [1, 'John', 'Smith']


def write_to_csv(ls):
    """
    :param ls: list argument to be written in CSV file
    List item will be written as a row, with every list value on a separate cell
    :return: None
    """
    with open("results.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(ls)